In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

In [24]:
def map_dual_to_commodity(dual):
    dual_name = {
    'gsl': 'Gasoline',
    'dsl': 'Diesel',
    'elc': 'Electricity',
    'h2': 'Gaseous H2',
    }
    for prefix, commodity in dual_name.items():
        if prefix in dual:
            return commodity
    return 'Other'

In [ ]:
def load_carrier_duals(db_variant='vanilla4_dual_carriers', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    # filter db tables  -- note the tolerance for absolute dual values WHERE abs(dual) > 1e-4 AND abs(dual) < 1e4
    query_built_capacity = """
        SELECT
            scenario,
            substr(constraint_name, instr(constraint_name, 'ON,') + 3, 4) AS period,
            substr(constraint_name, instr(constraint_name, ',T_') + 1, instr(constraint_name, ']') - instr(constraint_name, ',T_') - 1) AS commodity,
            AVG(dual) AS avg_dual
        FROM OutputDualVariable
        WHERE abs(dual) > 1e-4 AND abs(dual) < 1e4 
            AND constraint_name LIKE 'DemandConstraint[ON%' AND (
                constraint_name LIKE '%T_D_gsl]' OR
                constraint_name LIKE '%T_D_dsl]' OR
                constraint_name LIKE '%T_D_elc]' OR
                constraint_name LIKE '%T_D_h2]'
            )
        GROUP BY period, commodity
        ORDER BY commodity, period
        """
    
    df = pd.read_sql_query(query_built_capacity, conn)
    conn.close()
    df['commodity'] = df['commodity'].apply(map_dual_to_commodity)
    df['period'] = df['period'].astype(int)
    return df

In [26]:
load_carrier_duals(
    # db_variant='baseline_dual_carriers'
    )

,scenario,period,commodity,avg_dual
0,vanilla4_dual_carriers,2021,Diesel,119.136663
1,vanilla4_dual_carriers,2025,Diesel,99.124901
2,vanilla4_dual_carriers,2030,Diesel,78.226436
3,vanilla4_dual_carriers,2035,Diesel,69.888995
4,vanilla4_dual_carriers,2040,Diesel,61.500288
5,vanilla4_dual_carriers,2045,Diesel,53.852858
6,vanilla4_dual_carriers,2050,Diesel,47.393771
7,vanilla4_dual_carriers,2021,Electricity,477.792821
8,vanilla4_dual_carriers,2025,Electricity,471.288198
9,vanilla4_dual_carriers,2030,Electricity,290.854430


In [32]:
def build_master_df(db_variants, path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    all_parts = []

    for variant in db_variants:
        if 'dual_carriers' in variant:
            df_carriers = load_carrier_duals(db_variant=variant, path=path)
            df_carriers['comm_type'] = 'carrier'
            # annualize the duals
            df_carriers['A/P'] = df_carriers.apply(
                lambda row: row['avg_dual'] * 0.03 / (1 - (1 + 0.03)**(-4 if row['period'] == 2021 else -5)), axis=1
            )
            # put the cost back into real dollars
            df_carriers['Shadow Price'] = df_carriers['A/P'] * (1 + 0.03)**(df_carriers['period'] - 2021 - 1)
            all_parts.append(df_carriers)
        else:
            exception_msg = f"Unsupported db_variant: {variant}. Only dual variants supported."
            raise ValueError(exception_msg)            


    master_df = pd.concat(all_parts, ignore_index=True)
    return master_df

In [33]:
df_scenarios = build_master_df(
    db_variants=[
        'vanilla4_dual_carriers',
        'baseline_dual_carriers',
    ]
)

df_scenarios

,scenario,period,commodity,avg_dual,comm_type,A/P,Shadow Price
0,vanilla4_dual_carriers,2021,Diesel,119.136663,carrier,32.050985,31.117461
1,vanilla4_dual_carriers,2025,Diesel,99.124901,carrier,21.644375,23.651393
2,vanilla4_dual_carriers,2030,Diesel,78.226436,carrier,17.081100,21.637826
3,vanilla4_dual_carriers,2035,Diesel,69.888995,carrier,15.260582,22.410678
4,vanilla4_dual_carriers,2040,Diesel,61.500288,carrier,13.428869,22.861751
5,vanilla4_dual_carriers,2045,Diesel,53.852858,carrier,11.759018,23.207439
6,vanilla4_dual_carriers,2050,Diesel,47.393771,carrier,10.348647,23.676955
7,vanilla4_dual_carriers,2021,Electricity,477.792821,carrier,128.539191,124.795331
8,vanilla4_dual_carriers,2025,Electricity,471.288198,carrier,102.907932,112.450276
9,vanilla4_dual_carriers,2030,Electricity,290.854430,carrier,63.509394,80.451801


In [41]:
color_fuel_map = {
    'Gasoline': 'red',
    'Diesel': 'brown',
    # 'Compressed NG': 'tomato',
    # 'Hybrid': 'darkorange',
    # 'PHEV (35-mile AER)': 'dodgerblue',
    # 'PHEV (50-mile AER)': 'darkblue',
    # 'BEV (150-mile AER)': 'limegreen',
    # 'BEV (200-mile AER)': 'seagreen',
    # 'BEV (300-mile AER)': 'olive',
    # 'BEV (400-mile AER)': 'darkgreen',
    # 'Plug-in hybrid': 'blue',
    'Electricity': 'green',
    'Gaseous H2': 'mediumvioletred'
}

In [43]:
df_scenarios['scenario'] = df_scenarios['scenario'].replace({
    'vanilla4_dual_carriers': 'Vanilla',
    'baseline_dual_carriers': 'Baseline'
})

fig = px.line(df_scenarios, 
              x='period', 
              y='Shadow Price', 
              color='commodity', 
              color_discrete_map=color_fuel_map,
              line_dash='scenario',
              title='Annualized Shadow Prices of Carriers by Commodity and Period',
              labels={'Shadow Price': 'Annualized Shadow Price ($/GJ)', 'scenario': 'Scenario'},
              markers=True,
              template='plotly_white',
              category_orders={'scenario': ['Vanilla', 'Baseline']}
              )

fig.update_layout(
    xaxis_title='Period',
    yaxis_title='Annualized Shadow Price (in $/GJ)',
    legend_title_text='Commodity',
    height=600,
    width=1000,
)

fig.show()